In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import torch
from torch import nn
import random
import torchvision
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import json
from collections import namedtuple
from sklearn.metrics import classification_report
import cv2

In [6]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_data():
  resize = 224

  transform_train = transforms.Compose([
      
      transforms.Resize((resize, resize)),
      #Preprocess: Can bang histogram
      #transforms.RandomEqualize(p=0.5),


      # Augmentation:Xoay anh, bop meo anh
      #transforms.RandAugment(num_ops=3),
      #transforms.RandomAffine(degrees=90, translate=None, shear=25),

      transforms.ToTensor()
      #transforms.Normalize(mean, std)
  ])

  transform_test = transforms.Compose([
    
    transforms.Resize((resize, resize)),
    
    #transforms.RandomEqualize(p=0.2),
    #transforms.RandAugment(num_ops=3),
    # transforms.RandomAffine(degrees=45, translate=None, shear=15),
    
    transforms.ToTensor(),
    #transforms.Normalize(mean, std)
  ])

  trainset = torchvision.datasets.ImageFolder(root='./drive/MyDrive/data/DATA_CHAMBER_2021/train', transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root='./drive/MyDrive/data/DATA_CHAMBER_2021/test', transform=transform_test)

  print("Number of Image in train set:", len(trainset))
  print("Number of Image in test set:", len(testset))
  print("Classes: ", trainset.class_to_idx)

  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=32, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=32, shuffle=False, num_workers=4)
  print("Num batch in train set: ", len(trainloader))
  return TrainTest(train=trainloader, test=testloader)

def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 20
  step = 0
  for images, labels in loader:
    step += 1
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if step % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {step} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0

def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for images, labels in loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue



In [7]:
def main(MODEL='vgg16', type='img'):
  classes = get_classes()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0")
  use_pretrained = False

  if MODEL == 'vgg19':
    PATH='./vgg19.pth'
    model = models.vgg19(pretrained=use_pretrained)
    model.classifier[6] = nn.Linear(in_features=4096, out_features=3)
  elif MODEL == 'resnet50':
    PATH='./resnet50.pth'
    model = models.resnet50(pretrained=use_pretrained)
    model.fc = nn.Linear(in_features=2048, out_features=3)
  else:
    PATH='./vgg16.pth'
    model = models.vgg16(pretrained=use_pretrained)
    model.classifier[6] = nn.Linear(in_features=4096, out_features=3) 

  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)


  if type=='img':

    print(classification_report(ytrue, ypred, target_names=classes))
    torch.save(model.state_dict(), PATH)
    return model

  elif type=='video':
    ytrue = np.array([10])
    ypred = np.array([10])
    vid = -1
    true = 0
    count = np.array([0,0,0])
    predict = 0
    
    for i in range (len(datasets.test.imgs)):
      if (datasets.test.imgs[i][0].split('C/')[1].split('_')[0] != video):
        if ((count[0] != 0) | (count[1] != 0) | (count[2] != 0)):
          max_ = count.max()
          for j in range (len(count)):
            if (count[j] == max_):
              predict = j
              break 
          print('video:',vid, '  true:', true, '  predic:',predict)
          ytrue_ = np.append(ytrue_, true)
          ypred_ = np.append(ypred_, predict)
          count[:] = 0
        video = datasets.test.imgs[i][0].split('C/')[1].split('_')[0]
        true = ytrue[i] 
      count[ypred[i]] += 1
    print('video:',vid, '  true:', true, '  predic:',predic)
    ytrue_ = np.append(ytrue_, true)
    ypred_ = np.append(ypred_, predict)
    ytrue_ = np.delete(ytrue_, 0)
    ypred_ = np.delete(ypred_, 0)

    print(classification_report(ytrue_, ypred_, target_names=classes))



In [2]:
model_VGG16 = main(MODEL='vgg16')

NameError: ignored

In [ ]:
model_VGG19 = main(MODEL='vgg19')

Number of Image in train set: 6717
Number of Image in test set: 1607
Classes:  {'2C': 0, '3C': 1, '4C': 2}
Num batch in train set:  210


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0 step 19 ave_loss 1.0434
Epoch 0 step 39 ave_loss 1.0986
Epoch 0 step 59 ave_loss 1.0883
Epoch 0 step 79 ave_loss 1.0473
Epoch 0 step 99 ave_loss 1.1488
Epoch 0 step 119 ave_loss 1.0943
Epoch 0 step 139 ave_loss 1.0981
Epoch 0 step 159 ave_loss 1.0982
Epoch 0 step 179 ave_loss 1.1002
Epoch 0 step 199 ave_loss 1.0968


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          2C       0.00      0.00      0.00       409
          3C       0.23      1.00      0.37       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.23      1607
   macro avg       0.08      0.33      0.12      1607
weighted avg       0.05      0.23      0.08      1607



...

Epoch 7 step 19 ave_loss 0.0023
Epoch 7 step 39 ave_loss 0.0178
Epoch 7 step 59 ave_loss 0.0258
Epoch 7 step 79 ave_loss 0.0376
Epoch 7 step 99 ave_loss 0.0134
Epoch 7 step 119 ave_loss 0.0165
Epoch 7 step 139 ave_loss 0.0150
Epoch 7 step 159 ave_loss 0.0232
Epoch 7 step 179 ave_loss 0.0089
Epoch 7 step 199 ave_loss 0.0005
              precision    recall  f1-score   support

          2C       0.90      0.99      0.94       409
          3C       0.92      0.91      0.91       367
          4C       1.00      0.96      0.98       831

    accuracy                           0.95      1607
   macro avg       0.94      0.95      0.94      1607
weighted avg       0.96      0.95      0.95      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 8 step 19 ave_loss 0.0281
Epoch 8 step 39 ave_loss 0.0491
Epoch 8 step 59 ave_loss 0.0196
Epoch 8 step 79 ave_loss 0.0204
Epoch 8 step 99 ave_loss 0.0262
Epoch 8 step 119 ave_loss 0.0243
Epoch 8 step 139 ave_loss 0.0129
Epoch 8 step 159 ave_loss 0.0015
Epoch 8 step 179 ave_loss 0.0002
Epoch 8 step 199 ave_loss 0.0072
              precision    recall  f1-score   support

          2C       0.94      0.75      0.83       409
          3C       0.72      1.00      0.84       367
          4C       1.00      0.92      0.96       831

    accuracy                           0.90      1607
   macro avg       0.88      0.89      0.88      1607
weighted avg       0.92      0.90      0.90      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 9 step 19 ave_loss 0.0012
Epoch 9 step 39 ave_loss 0.0017
Epoch 9 step 59 ave_loss 0.0003
Epoch 9 step 79 ave_loss 0.0124
Epoch 9 step 99 ave_loss 0.0170
Epoch 9 step 119 ave_loss 0.0085
Epoch 9 step 139 ave_loss 0.0016
Epoch 9 step 159 ave_loss 0.0085
Epoch 9 step 179 ave_loss 0.0072
Epoch 9 step 199 ave_loss 0.0042
              precision    recall  f1-score   support

          2C       0.87      0.90      0.89       409
          3C       0.76      0.99      0.86       367
          4C       1.00      0.84      0.92       831

    accuracy                           0.89      1607
   macro avg       0.88      0.91      0.89      1607
weighted avg       0.91      0.89      0.90      1607



In [ ]:
model_Resnet50 = main(MODEL='resnet50')

Number of Image in train set: 6717
Number of Image in test set: 1607
Classes:  {'2C': 0, '3C': 1, '4C': 2}
Num batch in train set:  210


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0 step 19 ave_loss 5.0304
Epoch 0 step 39 ave_loss 2.0670
Epoch 0 step 59 ave_loss 1.2899
Epoch 0 step 79 ave_loss 1.1798
Epoch 0 step 99 ave_loss 1.3397
Epoch 0 step 119 ave_loss 1.1718
Epoch 0 step 139 ave_loss 1.1521
Epoch 0 step 159 ave_loss 1.0904
Epoch 0 step 179 ave_loss 1.0360
Epoch 0 step 199 ave_loss 1.0498
              precision    recall  f1-score   support

          2C       0.34      0.91      0.50       409
          3C       0.96      0.06      0.11       367
          4C       0.61      0.36      0.46       831

    accuracy                           0.43      1607
   macro avg       0.64      0.45      0.36      1607
weighted avg       0.62      0.43      0.39      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1 step 19 ave_loss 0.8248
Epoch 1 step 39 ave_loss 0.9841
Epoch 1 step 59 ave_loss 0.7679
Epoch 1 step 79 ave_loss 0.5917
Epoch 1 step 99 ave_loss 0.4852
Epoch 1 step 119 ave_loss 0.5661
Epoch 1 step 139 ave_loss 0.3966
Epoch 1 step 159 ave_loss 0.3378
Epoch 1 step 179 ave_loss 0.3250
Epoch 1 step 199 ave_loss 0.2749
              precision    recall  f1-score   support

          2C       0.60      0.70      0.65       409
          3C       0.35      0.57      0.43       367
          4C       0.77      0.50      0.61       831

    accuracy                           0.57      1607
   macro avg       0.58      0.59      0.56      1607
weighted avg       0.63      0.57      0.58      1607



...

Epoch 7 step 19 ave_loss 0.0044
Epoch 7 step 39 ave_loss 0.0039
Epoch 7 step 59 ave_loss 0.0092
Epoch 7 step 79 ave_loss 0.0113
Epoch 7 step 99 ave_loss 0.0051
Epoch 7 step 119 ave_loss 0.0060
Epoch 7 step 139 ave_loss 0.0037
Epoch 7 step 159 ave_loss 0.0010
Epoch 7 step 179 ave_loss 0.0063
Epoch 7 step 199 ave_loss 0.0005
              precision    recall  f1-score   support

          2C       0.75      0.96      0.85       409
          3C       0.65      0.72      0.68       367
          4C       0.96      0.79      0.87       831

    accuracy                           0.82      1607
   macro avg       0.79      0.82      0.80      1607
weighted avg       0.84      0.82      0.82      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 8 step 19 ave_loss 0.0002
Epoch 8 step 39 ave_loss 0.0010
Epoch 8 step 59 ave_loss 0.0063
Epoch 8 step 79 ave_loss 0.0052
Epoch 8 step 99 ave_loss 0.0012
Epoch 8 step 119 ave_loss 0.0032
Epoch 8 step 139 ave_loss 0.0026
Epoch 8 step 159 ave_loss 0.0012
Epoch 8 step 179 ave_loss 0.0009
Epoch 8 step 199 ave_loss 0.0025
              precision    recall  f1-score   support

          2C       0.80      0.79      0.79       409
          3C       0.54      0.73      0.62       367
          4C       0.93      0.79      0.85       831

    accuracy                           0.78      1607
   macro avg       0.76      0.77      0.76      1607
weighted avg       0.81      0.78      0.79      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 9 step 19 ave_loss 0.0010
Epoch 9 step 39 ave_loss 0.0001
Epoch 9 step 59 ave_loss 0.0001
Epoch 9 step 79 ave_loss 0.0022
Epoch 9 step 99 ave_loss 0.0016
Epoch 9 step 119 ave_loss 0.0007
Epoch 9 step 139 ave_loss 0.0004
Epoch 9 step 159 ave_loss 0.0001
Epoch 9 step 179 ave_loss 0.0000
Epoch 9 step 199 ave_loss 0.0001
              precision    recall  f1-score   support

          2C       0.81      0.83      0.82       409
          3C       0.58      0.79      0.67       367
          4C       0.95      0.78      0.86       831

    accuracy                           0.80      1607
   macro avg       0.78      0.80      0.78      1607
weighted avg       0.83      0.80      0.81      1607



In [ ]:
video = main(MODEL='vgg16', type='video')

Number of Image in train set: 6717
Number of Image in test set: 1607
Classes:  {'2C': 0, '3C': 1, '4C': 2}
Num batch in train set:  210


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0 step 19 ave_loss 1.0477
Epoch 0 step 39 ave_loss 1.0921
Epoch 0 step 59 ave_loss 1.0432
Epoch 0 step 79 ave_loss 1.1580
Epoch 0 step 99 ave_loss 1.0959
Epoch 0 step 119 ave_loss 1.0937
Epoch 0 step 139 ave_loss 1.0826
Epoch 0 step 159 ave_loss 1.0272
Epoch 0 step 179 ave_loss 1.0871
Epoch 0 step 199 ave_loss 1.0036
Epoch 1 step 19 ave_loss 1.0434
Epoch 1 step 39 ave_loss 1.0938
Epoch 1 step 59 ave_loss 1.0977
Epoch 1 step 79 ave_loss 1.1005
Epoch 1 step 99 ave_loss 1.0950
Epoch 1 step 119 ave_loss 1.0917
Epoch 1 step 139 ave_loss 1.1024
Epoch 1 step 159 ave_loss 1.0982
Epoch 1 step 179 ave_loss 1.1012
Epoch 1 step 199 ave_loss 1.0988
Epoch 2 step 19 ave_loss 1.0426
Epoch 2 step 39 ave_loss 1.0927
Epoch 2 step 59 ave_loss 1.0967
Epoch 2 step 79 ave_loss 1.0970
Epoch 2 step 99 ave_loss 1.0911
Epoch 2 step 119 ave_loss 1.0838
Epoch 2 step 139 ave_loss 1.0730
Epoch 2 step 159 ave_loss 0.8238
Epoch 2 step 179 ave_loss 0.6930
Epoch 2 step 199 ave_loss 0.5983
Epoch 3 step 19 ave_loss 